### 1. Upload dataset (.dta)

In [1]:
use "https://github.com/worldbank/Stata-IE-Visual-Library/raw/develop/Library/Bar%20plots/Bar%20plot%20of%20multiple%20variables/data.dta", clear

### 2.  We will create mean of all crop observations across each ``treatment_group``, baseline, and midline category. 
First we collapse baseline and midline variables to get Treatment/Control mean.

In [2]:
collapse bl_* ml_*, by(treatment_group) 

variable treatment_group not found
(error in option by())


r(111);


### 3. Reshape dataset to a long-format, indexed by the new "crop" variable. This is a string variable with the name of the crop.

(e.g. variables ``ml_w_Barley_Prod`` and ``bl_w_Barley_Prod``, ``w_Barley_Prod`` becomes the value of the new `crop` variable.) 


End product: data with columns of 

    treatment_group
    crop
    bl_ 
    ml_

In [ ]:
qui reshape long bl_ ml_, i(treatment_group) j(crop) str

### 4. Reshape again dataset to wide-format, indexed by "treatment_group" for each baseline and midlline values.

For each value of ``crop``, the code takes variable ``treatment_group``, and creates merged variable with ``bl_`` and ``ml_`` variables.
(e.g. Each ``bl_`` variable will now have two types, ``bl_0 `` and ``bl_1``, using values from ``treatment_group``, which is 0 and 1.)

Data will have columns of:

    crop
    bl_0
    bl_1
    ml_0
    ml_1

In [ ]:
qui reshape wide bl_ ml_, i(crop) j(treatment_group)

### 5. Keeping only the variables of interests

In [ ]:
gsort -ml_1
qui keep if _n<4

### 6. Creating position vector for bars for the y-axis

In [ ]:
qui gen xml_1 = _n
qui gen xml_0 = xml_1 +.2
qui gen xbl_1 = xml_1 +.4
qui gen xbl_0 = xml_1 +.6

### 7. Cleaning variables for the graph

In [ ]:
qui foreach var of varlist bl_* ml_* {
	replace `var' = round(`var',.1)
	}

qui replace crop = subinstr(crop, "_", " ",.) 
qui replace crop = subinstr(crop, "w", " ",.)

### 8. Inserting the list of values in the local macro macname 

In [ ]:
qui forvalues j = 1/3 {
	levelsof crop if _n==`j', local(crop`j')
	}

### 9. Creating a horizontal bar graph with cleaned data

In [ ]:
twoway bar bl_0 xbl_0, horizontal bcolor(edkblue) barwidth(.2) || ///
	bar ml_0 xml_0, horizontal bcolor(eltblue) barwidth(.2) || ///
	scatter xml_0 ml_0, mlabel(ml_0) mlabcolor(black) msymbol(none) mlabsize(vsmall) || ///
	scatter xbl_0 bl_0, mlabel(bl_0) mlabcolor(black) msymbol(none) mlabsize(vsmall) || ///
	bar bl_1 xbl_1, horizontal bcolor(emidblue) barwidth(.2) || ///
	bar ml_1 xml_1, horizontal bcolor(black) barwidth(.2) || ///
	scatter xml_1 ml_1, mlabel(ml_1) mlabcolor(black) msymbol(none) mlabsize(vsmall) || ///
	scatter xbl_1 bl_1, mlabel(bl_1) mlabcolor(black) msymbol(none) mlabsize(vsmall) ///
	xlabel(0 "0" 50 "50" 100 "100" 150 "150") ///
	xscale(range(0(20)100)) /// 
	ylabel(1.3 `crop1' 2.3 `crop2' 3.3 `crop3', angle(0)) ///
	graphregion(color(white)) bgcolor(white) ///
	legend(label(1 "Baseline-Control") label(2 "Midline-Control") label(5 "Baseline-Treatment") label(6 "Midline-Treatment") order(1 5 2 6)) 

### 10. Exporting the graph

In [ ]:
graph export "figure.png", replace